+ Pixelized shear measurements (smoothing -> pixelize/ pixelize -> smoothing)
+ Get the variance map (mock/analytical)

In [1]:
%pylab inline
import os
import gc
import glob
import numpy as np
import astropy.io.fits as pyfits
import maputil as mapUtil
from configparser import ConfigParser
from pixel3D import cartesianGrid3D
from scipy.stats import norm as statnorm

Populating the interactive namespace from numpy and matplotlib


In [11]:
configName  =   'wrkDir/config-nl15.ini'
parser      =   ConfigParser()
parser.read(configName)

configName  =   'wrkDir/config-nl15.ini'
pDict={'dempz': 'demp',\
       'dnnz': 'dnn',\
       'mizuki': 'mizuki'}

def plotHist(fieldname,array,pixels,oname):
    cmap=plt.get_cmap('tab20')

    plt.figure(figsize=(8,6))

    gbin=plt.hist(array,bins=100,density=True,range=(-1.2,1.2),histtype='step',label='galaxy',color=cmap(0))[1]
    gbinGal=(gbin-np.average(array))/np.std(array)
    a=statnorm.pdf(gbinGal)
    plt.plot(gbin,a/np.sum(a)/(gbin[1]-gbin[0]),color=cmap(0),ls='--')

    gbin=plt.hist(pixels,bins=100,density=True,range=(-1.2,1.2),histtype='step',label='pixel',color=cmap(2))[1]

    gbinPix=(gbin-np.average(pixels))/np.std(pixels)
    a=statnorm.pdf(gbinPix)
    plt.plot(gbin,a/np.sum(a)/(gbin[1]-gbin[0]),color=cmap(2),ls='--')
    plt.title('%s' %fieldname,fontsize=20)
    plt.xlabel(r'$\delta g_1$',fontsize=20)
    plt.ylabel(r'$P(\delta g_1)$',fontsize=20)
    plt.legend(fontsize=20)
    plt.yscale('log')
    plt.ylim(1e-4,15)
    plt.grid()
    plt.tight_layout()
    plt.savefig(oname)
    plt.close()
    return

In [12]:
key='dempz'
colname=pDict[key]
for fieldname in mapUtil.field_names:
    inDir  =   'wrkDir/reGaus_catalog/%s_groups/' %fieldname
    outDir =   'wrkDir/reGaus_pixel3D/%s_groups/' %fieldname
    if not os.path.isdir(outDir):
        os.mkdir(outDir)
    fnames=glob.glob(os.path.join(inDir,'src-group-*.fits'))
    for fname in fnames:
        gn=fname.split('-')[-1].split('.')[0]
        # update zbounds
        bname=os.path.join(outDir,'group-%s-%s-10bins.fits' %(gn,key))
        bounds=pyfits.getdata(bname,ext=1)['zbounds']
        parser.set('sourceZ','zbound','[%s]' %(','.join([str(bb) for bb in bounds])))
        gridInfo    =   cartesianGrid3D(parser)
        glab=fieldname+"-%s"%gn
        # poz average
        data=pyfits.getdata(fname)
        mask=(~np.isnan(data['%s_zbest' %colname]))\
            &(data['%s_zbest_std'%colname]<1.0)\
            &(data['%s_zbest'%colname]<3.0)\
            &(data['%s_zbest'%colname]>0.05)
        data=data[mask]
        wsAve=np.average(data['weight'])
        ws=data['weight']/wsAve
        z=data['%s_zbest'%colname]
        x,y=gridInfo.setupTanPlane(data['ra'],data['dec'])
        out1=gridInfo.pixelize_data(x,y,z,data['g1'],ws=ws)
        out2=gridInfo.pixelize_data(x,y,z,data['g2'],ws=ws)
        outname1=os.path.join(outDir,'group-%s-%s-g1Map.fits' %(gn,key))
        outname2=os.path.join(outDir,'group-%s-%s-g2Map.fits' %(gn,key))
        pyfits.writeto(outname1,out1,overwrite=True)
        pyfits.writeto(outname2,out2,overwrite=True)
        outname3=os.path.join(outDir,'group-%s-%s-g1Hist.png' %(gn,key))
        plotHist(glab,data['g1'],out1[out1!=0],outname3)
        del out1,out2,data,bounds,x,y,z,ws
        gc.collect()